In [ ]:
#importing all the needed packages and libraires
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import pandas as pd
#import tflearn
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import random
import tensorflow

In [ ]:
df = pd.read_json("C:/Users/shiva/Downloads/json file/intents.json") #loading the data

In [ ]:
stmr = LancasterStemmer() #creating an empty variables but are done so that we can store the analyzied 
words = []                #data into it after undergoing various analysis and techniques on the data 
labels = []
docs_x = []
docs_y = []

In [ ]:
#performing looping statements and conditional statmnts in order to clean and bring out the needed/fine information
#includes data cleaning lowering cases data conversion

for intent in df["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
words = [stmr.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)
training = []
output = []
out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag = []
    wrds = [stmr.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)
training = np.array(training)
output = np.array(output)

In [ ]:
#aplying deep learning techniques
#creating hidden layers applying activation functions

k = Sequential()
k.add(Dense(output_dim=8,init ='he_uniform',activation='relu',input_dim =(len(training[0]))))
k.add(Dropout(0.5))
k.add(Dense(output_dim=8,init ='he_uniform',activation='relu'))
k.add(Dropout(0.5))
k.add(Dense(output_dim=len(output[0]),init ='he_uniform',activation='softmax'))

In [ ]:
#performing a bit of tuning for the better performance

sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
k.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])

In [ ]:
#fitting the trainied model to the data and itirating throughs needed epochs to find out the best accuracy

k.fit(training,output,batch_size=5,nb_epoch=50)

In [ ]:
#defing funtion so to convert the user information and includes 
#data convertion lowering cases conditional/looping statmnts 

def bag_of_words(s,words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stmr.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1
    return np.array(bag)

In [ ]:
def chat():
    print("start talking with bot (type quit to stop)")
    while True:
        inp = input("you: ")
        if inp.lower()=="quit":
            break
        results = k.predict([bag_of_words(inp,words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in df["intents"]:
            if tg["tag"] == tag:
                responses = tg["responses"]
        print(random.choice(responses))
#chat()

In [ ]:
chat()